In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# General imports.
import ipywidgets as ipw
from IPython.display import clear_output
import numpy as np

# AiiDA imports.
%aiida
from aiida.engine import submit, run, run_get_node

# AiiDA lab imports.
from aiidalab_widgets_base import CodeDropdown, SubmitButtonWidget, StructureBrowserWidget

# Local imports
from apps.surfaces.reactions.dft_details_dev import DFTDetails
from apps.surfaces.reactions.neb_details import NebDetails
from apps.surfaces.reactions.viewer_details import ViewerDetails
from apps.surfaces.reactions.slab_validity import slab_is_valid
from apps.surfaces.reactions.suggested_param import suggested_parameters
from apps.surfaces.reactions import analyze_structure
from apps.surfaces.reactions.metadata import MetadataWidget
from apps.surfaces.reactions.workchains import NEBWorkChain

In [ ]:
# General parameters.
job_details = {'workchain':'NEBWorkChain'}
atoms = []
details = []
slab_analyzed=[]

In [ ]:
# Widgets monitor functions.
def on_struct_change(c):
    global atoms, slab_analyzed
    if not struct_browser.results.value:
        return
    job_details['structure']=struct_browser.results.value
    node = struct_browser.results.value
    atoms = node.get_ase()
    atoms.pbc = [1, 1, 1]
    slab_analyzed = analyze_structure.analyze(atoms)
    job_details['slab_analyzed']=slab_analyzed
    dft_details_widget.reset()  
    neb_details_widget.reset()
    guess_calc_params()
    
    viewer_widget.setup(atoms, slab_analyzed)
    
    with mol_ids_info_out:
        clear_output()
        print(slab_analyzed['summary'])

def on_fixed_atoms_btn_click(c):
    if dft_details_widget.btn_fixed_pressed:
        viewer_widget.show_fixed(dft_details_widget.fixed_atoms.value)
    else:
        viewer_widget.show_fixed("")
        
def guess_calc_params():
    method=dft_details_widget.calc_type.value
    valid_slab,msg = slab_is_valid(slab_analyzed,method)
    if valid_slab:        
        atoms_to_fix,num_nodes=suggested_parameters(slab_analyzed,method)
        dft_details_widget.reset(fixed_atoms=atoms_to_fix,calc_type=method)
        num_nodes_selector.value = num_nodes
    else:
        print(msg)

# Submit a NEB calculation

## Structure

In [ ]:
# Display widgets and define job parameters.

# Strciture selection.
struct_browser = StructureBrowserWidget()
struct_browser.results.observe(on_struct_change, names='value')

# Displaying the structure viewer.
viewer_widget = ViewerDetails()
mol_ids_info_out = ipw.Output()
display(ipw.VBox([struct_browser, viewer_widget, mol_ids_info_out]))

# Code selection.
computer_code_dropdown = CodeDropdown(input_plugin='cp2k', path_to_root="../../")

# Misc details.
style = {'description_width': '120px'}
layout = {'width': '70%'}
num_nodes_selector = ipw.IntText(
    value=1,
    description='# Nodes',
    style=style, layout=layout)
calc_name_selector = ipw.Text(
    description='Calculation Name: ',
    placeholder='A great name.',
    style=style, layout=layout)

# DFT.
dft_details_widget = DFTDetails(job_details=job_details)    
dft_details_widget.btn_fixed_atoms.on_click(on_fixed_atoms_btn_click)
dft_details_widget.calc_type.observe(lambda c: guess_calc_params(), names='value')

# NEB.
neb_details_widget = NebDetails(computer_code_dropdown, dft_details_widget) 
def enable_submit(c):
    btn_submit.btn_submit.disabled=False
neb_details_widget.setup_btn.on_click(enable_submit)

display(
    ipw.VBox(
        [
            computer_code_dropdown,
            num_nodes_selector,
            calc_name_selector,
            dft_details_widget,
            neb_details_widget
        ]
    )
)

In [ ]:
def get_plain_input():
    error_msg, submit_dict = CP2K2DICT(input_lines=plain_input.value)
    if error_msg != "":
        print(error_msg)
    else:
        input_builder = Cp2kBaseWorkChain.get_builder()
        
        # Code.
        input_builder.cp2k.code = computer_code_dropdown.selected_code
        # Structure.
        input_builder.cp2k.file.input_xyz = make_geom_file(structure_selector.structure_node, Str("mol_on_slab.xyz")) 
        if details_dict['calc_type'] != 'Full DFT':
            # slab potential
            slab_element=list(slab_analyzed['slab_elements'])[0]
            input_builder.cp2k.file.pot_f = SinglefileData(file='/home/aiida/apps/surfaces/slab/'+slab_element+'.pot')
            mol_indexes = list(itertools.chain(*slab_analyzed['all_molecules']))
            input_builder.cp2k.file.mol_xyz = make_geom_file(structure_selector.structure_node,
                                                             Str("mol.xyz"),
                                                             selection=List(list=mol_indexes))
        input_builder.handler_overrides = Dict(dict={'resubmit_unconverged_geometry':True})
        input_builder.cp2k.metadata = metadata_widget.dict
        input_builder.metadata.label = "SlabGeoOptWorkChain"
        input_builder.metadata.description = process_description.value
        input_builder.cp2k.metadata['label'] = "slab_geo_opt"
        input_builder.cp2k.metadata['description'] = process_description.value
        input_builder.cp2k.parameters = Dict(dict=submit_dict)
    return input_builder

In [ ]:
def build_inputs():
    builder = NEBWorkChain.get_builder()
    builder.cp2k_code = computer_code_dropdown.selected_code
    builder.structure = struct_browser.results.value
    builder.max_force = Float(dft_details_widget.max_force.value)
    builder.calc_type = Str(dft_details_widget.calc_type.value)
    builder.vdw_switch = Bool(dft_details_widget.vdw_switch.value)
    builder.mgrid_cutoff = Int(dft_details_widget.mgrid_cutoff.value)
    builder.fixed_atoms = Str(dft_details_widget.fixed_atoms.value)
    builder.num_machines = Int(num_nodes_selector.value)
    builder.struc_folder = neb_details_widget.struct_folder
    builder.wfn_cp_commands = List(list=neb_details_widget.aiida_wfn_cp_list)
    builder.calc_name = Str(calc_name_selector.value)
    builder.nproc_rep = Int(neb_details_widget.proc_rep.value)
    builder.nreplicas = Int(neb_details_widget.num_rep.value)
    builder.replica_pks = Str(neb_details_widget.text_replica_pks.value)
    builder.spring = Float(neb_details_widget.spring_constant.value)
    builder.rotate = Bool(neb_details_widget.rotate_frames.value)
    builder.align = Bool(neb_details_widget.align_frames.value)
    builder.nstepsit = Int(neb_details_widget.nsteps_it.value)
    builder.endpoints = Bool(neb_details_widget.optimize_endpoints.value)
    builder.metadata = {
        "description": calc_name_selector.value,
        "label": "NEBWorkChain",
    }
    return builder

## Submission

In [ ]:
# Submit button.
btn_submit = SubmitButtonWidget(NEBWorkChain, input_dictionary_function=build_inputs)
btn_submit.btn_submit.disabled=True
display(btn_submit)